In [4]:
from urllib.parse import urlparse, urldefrag, urljoin
from urllib.request import urlopen
from bs4 import BeautifulSoup
from queue import Queue
import time as time

In [5]:
!pip install url_normalize

In [6]:
from url_normalize import url_normalize

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# from google.colab import files

In [41]:
our_url = 'https://ailita-shop.ru/catalog/semena/semena_ovoshchey/perets/perets'

columns = ['Название', 'Срок созревания', 'Толщина стенки', 'Высота растения', 'Высота']
link_0 = "https://ailita-shop.ru/catalog/semena/semena_ovoshchey/perets/"
link_page = 'https://ailita-shop.ru/catalog/semena/semena_ovoshchey/perets?PAGEN_1='
plant_name = 'perets'
NUM_PAGES = 6

In [8]:
def download_from_the_internet(url):
    '''Скачивает сраницу с интернета

    Параметры:
        url (str) - ссылка на страницу для скачивания

    Возвращает:
        str - html-страница в виде строки, None в случае неудачи
    '''
    try:
        return urlopen(url).read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        return None

    
def extract_links_from_html(url, html):
    '''Парсит ссылки на странице

    Принимает:
        url (str) - исходный урл страницы
        html (str) - содержание html-страницы

    Возвращает:
        list - список ссылок, находящихся на странице
    '''
    parser = BeautifulSoup(html)
    # Формируем ссылки на те страницы, на которые ссылается документ
    return [urljoin(url, link.get('href')) for link in parser.findAll('a')]


def extract_text_info_from_html(html):
    '''Парсит текстовую информацию на странице

    Принимает:
         html (str) - содержание html-страницы

    Возвращает:
        dict - текстовая часть страницы по ключу text,
               название по ключу title
    '''
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()
    
    # Объединяем строки текста
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    # Находим название на странице
    title = soup.find('title').string
    
    return {'text': text, 'title': title}

def load_web_pages(seed, max_downloads, filtration_function):
    '''Обходит web-страницы в ширину и загружает информацию о них.
    
    Принимает:
        seed (str) -- страница, с которой начинать обход.
        max_downloads (int) -- максимальное число загруженных страниц.
        filtration_function (str -> bool) -- функция, указывающая, 
            стоит ли загружать страницу. Пример: is_wiki_article.
            
    Возвращает:
        pages_json (list) - список словарей с информацией о страницах.
    '''
    
    # Создаём список со страницами
    pages_json = []
    
    # Создаём очередь для обхода в ширину
    q = Queue()
    q.put(seed)

    already_visited = set()
    n_downloads = 0
    time_start = time.time()

    while not q.empty():
        # Нормализуем урл
        main_url = url_normalize(q.get())
        if main_url in already_visited:
            continue
        already_visited.add(main_url)
        html = download_from_the_internet(main_url)

        # Извлекаем ссылки из страницы
        if not(html is None):
            children_links = extract_links_from_html(main_url, html)
            time.sleep(1)

            # Извлекаем текст страницы
            text_info = extract_text_info_from_html(html) 

            # Добавляем запись в таблицу
            text_info['url'] = main_url
            pages_json.append(text_info)

            n_downloads += 1
            if n_downloads > max_downloads:
                break

            # Добавляем ещё не посещённые ссылки в очередь
            for child in children_links:
                if url_normalize(child) not in already_visited \
                and filtration_function(child):
                    q.put(child)
                
    return pages_json


def is_article(url, checked):
    '''Проверяет, является ли ссылка страницей на simple.wikipedia'''
    if (checked in url):
        return True
    return False

In [9]:
def check_integer(string):
  return any(map(str.isdigit, string))

In [10]:
def delete_ptr(name):
  '''clear name without , '''
  while (',' in name):
    name = name[ :name.find(',')]
  return name

In [11]:
def name_and_hybrid(text):
  ''' returns name and bool - true if hybrid'''
  name = ''
  hybrid = False
  description = text[: text.find('семена |')]

  if 'F1' in description:
    hybrid = True
    name = description[ :description.find('F1') - 1]
    delete_ptr(name)
    return name, hybrid
  
  if 'шт' in description:
    last_index = description.find('шт')
    while(description[last_index] != ' '):
        last_index -=1
    
    name = description[ :last_index]
    return delete_ptr(name), hybrid
    

  index = description.find('г,') - 1
  if 'г,' in description and check_integer(description[index]):
    while(description[index] != ' '):
        index -=1
    name = description[ :index]
    return delete_ptr(name), hybrid
  
  return delete_ptr(name), hybrid

In [12]:
def weight(text):
  splited = text.split('\n')
  if 'Масса плода:' in splited:
    weight = splited[splited.index('Масса плода:') + 1]
    if not check_integer(weight) or len(weight) > 20:
      weight = None

    return weight

In [13]:
def ripening(text):
  splited = text.split('\n')
  feature = 'Срок созревания:'
  if feature in splited:
    rippening = splited[splited.index(feature) + 1]
    if not 'спел' in rippening:
      rippening = None
    return rippening

In [14]:
def ripening_days(text):
  last_index = text.find(' дней')
  if last_index == -1:
    last_index = text.find(' дня')

  if last_index == -1:
    last_index = text.find(' день')

  if last_index == -1:
    last_index = text.find(' дн')

  days = ''
  if last_index != -1:
    substr = text[:last_index]
    first_index = last_index - 1
    while substr[first_index] != ' ':
      first_index -=1
    days = substr[first_index + 1: last_index]
  else:
    days = None
  
  if check_integer(days):
    return days
  else:
    return None

In [15]:
def determ(text):
  splited = text.split('\n')
  feature = 'Тип роста:'
  if feature in splited:
    rippening = splited[splited.index(feature) + 1]
    if not 'детерминант' in rippening.lower():
      rippening = None
    return rippening

In [16]:
def height(text):
  splited = text.split('\n')
  feature = 'Высота:'
  if feature in splited:
    height_v = splited[splited.index(feature) + 1]
    if not check_integer(height_v) or len(height_v) > 20:
      height_v = None
    return height_v

In [17]:
def form(text):
    splitted = text.split('\n')
    feature = 'Форма растения:'
    form = None
    if feature in splitted:
        form = splitted[splitted.index(feature) + 1]
        if len(form) > 100:
            form = None
    return form

In [18]:
def type_of_kabachok(text):
    splitted = text.split('\n')
    feature = 'Вид кабачка:'
    type = None
    if feature in splitted:
        type = splitted[splitted.index(feature) + 1]
        if len(type) > 100:
            type = None
    return type

In [19]:
def color(text):
    splitted = text.split('\n')
    feature = 'Цвет кабачка:'
    color = None
    if feature in splitted:
        color = splitted[splitted.index(feature) + 1]
        if len(color) > 100:
            color = None
    return color

In [30]:
def parse_feature(text, feature):
    splitted = text.split('\n')
    extracted = None
    feature_tech = feature + ':'
    if feature_tech in splitted:
        extracted = splitted[splitted.index(feature_tech) + 1]
        if len(extracted) > 100:
            extracted = None
    return extracted

In [31]:
def clear_parser(aelita_df, columns, checked_url):
  seeds_df = pd.DataFrame(columns = columns)
  for description, url in zip(aelita_df['text'], aelita_df['url']):
    if checked_url not in url:
      continue
    
    name, hybrid = name_and_hybrid(description)
    row = pd.DataFrame([[
        name,
        *[parse_feature(description, feature) for feature in columns[1:]]
    ]], columns = columns)
    seeds_df = pd.concat([seeds_df, row], axis = 0,ignore_index=True)
  
  return seeds_df

In [32]:
def parse(columns, PAGES_SITE, clear_parser, link_0, link_page, plant_name):
  seeds_df = pd.DataFrame(columns =columns)
  aelita_df =  pd.DataFrame(columns =['text', 'title', 'url'])
  for i in range(PAGES_SITE):
    SEED = ''
    if i == 0:
      SEED = link_0
    else:
      SEED = link_page + str(i + 1)

    pages_number = 30
    json_data = load_web_pages(SEED, pages_number, lambda x: is_article(x, our_url))
    aelita_df = pd.concat([pd.DataFrame(json_data), aelita_df], axis = 0,ignore_index=True)
    seeds_df = pd.concat([clear_parser(pd.DataFrame(json_data), columns, our_url).drop_duplicates(), seeds_df], axis = 0,ignore_index=True)
  
  
    aelita_df.to_csv(f'text_{plant_name}_aelita.csv')
  # files.download('text_tomato_aelita.csv')

    seeds_df.to_csv(f'parsed_{plant_name}_aelita.csv')
  # files.download('parsed_tomato_aelita.csv')

In [42]:
parse(columns, NUM_PAGES, clear_parser, link_0, link_page, plant_name)

In [43]:
df = pd.read_csv(f'parsed_{plant_name}_aelita.csv', index_col=[0])
df.head()

,Название,Срок созревания,Толщина стенки,Высота растения,Высота
0,Перец острый Медвежий коготь,Раннеспелый,Тонкостенный,Среднерослое,NaN
1,Перец острый Китайский фонарик,NaN,Толстостенный,Высокорослое,130-160 см
2,Перец сладкий Д'Артаньян,Раннеспелый,Тонкостенный,Среднерослое,NaN
3,Перец острый Хвандо,Раннеспелый,Тонкостенный,Высокорослое,140-155 см
4,Перец сладкий Паприка,Раннеспелый,Тонкостенный,Среднерослое,NaN


In [ ]:
def helper_word(text, feature, condition):
  splited = text.split('\n')
  if feature in splited:
    filling = splited[splited.index(feature) + 1]
    if not condition in filling.lower():
      filling = None
    return filling

In [ ]:
def helper_integer(text, feature, length):
  splited = text.split('\n')
  if feature in splited:
    filling = splited[splited.index(feature) + 1]
    if not check_integer(filling) or len(filling) >= length:
      filling = None
    return filling

In [ ]:
aelita_df = pd.read_csv(f'text_{plant_name}_aelita.csv')
aelita_df['text'] = aelita_df['0']
aelita_df = aelita_df['text'] 

In [ ]:
text = aelita_df[290]

name = ''
hybrid = False
description = text[: text.find('семена |')]

if 'F1' in description:
  hybrid = True
  name = description[ :description.find('F1') - 1]
  print('F1',delete_ptr(name), hybrid)

if 'шт' in description:
  last_index = description.find('шт')
  while(description[last_index] != ' '):
      last_index -=1
  
  name = description[ :last_index]
  print('шт',delete_ptr(name), hybrid)
  

index = description.find('г,') - 1
if 'г,' in description and check_integer(description[index]):
  while(description[index] != ' '):
      index -=1
  name = description[ :index]
  print('г',delete_ptr(name), hybrid)

In [ ]:
name_and_hybrid(aelita_df[290])


In [ ]:
name
delete_ptr(name), hybrid

In [ ]:
df = pd.read_csv(f'parsed_{plant_name}_aelita.csv', index_col=[0])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop_duplicates()
df.to_csv(f'parsed_{plant_name}_aelita.csv')